In [1]:
# import package
from scipy.spatial.distance import pdist, squareform
from minepy import MINE
from sklearn.preprocessing import StandardScaler
from scipy import stats
import pandas as pd
import scipy
import numpy as np

In [2]:
# Import feature data
file_path = r"PF_data_lowvar.csv"
all_data = pd.read_csv(file_path)
data = all_data.iloc[:,all_data.columns != "ID" ]
descriptor_data = data.iloc[:,data.columns != "pf_log"]
all_data_name_list = list(all_data)
descriptor_name_list = list(descriptor_data)
descriptor_count = len(descriptor_name_list)

scaler = StandardScaler()
data_scaler = scaler.fit_transform(data)
DataFrame_data_scaler = pd.DataFrame(data_scaler)

In [3]:
data

,pf_log,T,spacegroup,a,b,c,alpha,beta,gamma,volume,...,min:num_s_valence,min:period,min:specific_heat,min:thermal_conductivity,min:vdw_radius,min:vdw_radius_alvarez,min:vdw_radius_mm3,min:vdw_radius_uff,min:sound_velocity,min:Polarizability
0,-1.296850,1000,187,3.365604,3.365604,3.063359,90.000000,90.000000,120.000007,30.050702,...,2,2,0.284,0.02583,155,166,193,334.5,333.600000,1.10
1,7.452482,1000,160,4.375164,4.375164,4.375164,57.826728,57.826728,57.826722,56.264958,...,2,4,0.201,3.00000,206,199,244,428.0,2610.000000,5.50
2,5.546150,1000,216,4.475784,4.475784,4.475784,60.000000,60.000000,60.000000,63.400440,...,1,4,0.237,0.12000,185,186,222,314.8,2467.612558,3.05
3,-4.584891,1000,216,3.214208,3.214208,3.214208,60.000000,60.000000,60.000000,23.480470,...,2,2,0.372,0.02583,155,166,193,366.0,333.600000,1.10
4,3.644443,1000,225,3.588021,3.588021,3.588021,60.000000,60.000000,60.000000,32.662539,...,2,2,0.180,0.02583,155,166,193,352.0,333.600000,1.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,-4.404720,500,216,3.935348,3.935348,3.935348,60.000000,60.000000,60.000000,43.095737,...,2,2,0.192,0.02583,155,166,193,360.6,333.600000,1.10
398,7.618568,500,216,3.261481,3.261481,3.261481,60.000000,60.000000,60.000000,24.531798,...,2,2,0.131,88.00000,170,177,204,312.0,4940.000000,1.67
399,0.152674,500,216,3.727867,3.727867,3.727867,60.000000,60.000000,60.000000,36.632471,...,2,2,0.128,0.02583,155,166,193,366.0,333.600000,1.10
400,-1.499308,500,216,4.327494,4.327494,4.327494,60.000000,60.000000,60.000000,57.305232,...,2,3,0.128,0.23600,180,190,222,414.7,818.000000,3.63


In [4]:
# Set the correlation coefficient parameter
data_pearson = DataFrame_data_scaler.corr(method = 'pearson')
data_spearman = DataFrame_data_scaler.corr(method = 'spearman')
mine = MINE(alpha=0.6, c=15)


# Distance correlation coefficient function definition
def distcorr(X, Y):
    X = np.atleast_1d(X)
    Y = np.atleast_1d(Y)
    if np.prod(X.shape) == len(X):
        X = X[:, None]
    if np.prod(Y.shape) == len(Y):
        Y = Y[:, None]
    X = np.atleast_2d(X)
    Y = np.atleast_2d(Y)
    n = X.shape[0]
    if Y.shape[0] != X.shape[0]:
        raise ValueError('Number of samples must match')
    a = squareform(pdist(X))
    b = squareform(pdist(Y))
    A = a - a.mean(axis=0)[None, :] - a.mean(axis=1)[:, None] + a.mean()
    B = b - b.mean(axis=0)[None, :] - b.mean(axis=1)[:, None] + b.mean()

    dcov2_xy = (A * B).sum() / float(n * n)
    dcov2_xx = (A * A).sum() / float(n * n)
    dcov2_yy = (B * B).sum() / float(n * n)
    dcor = np.sqrt(dcov2_xy) / np.sqrt(np.sqrt(dcov2_xx) * np.sqrt(dcov2_yy))
    return dcor

column_descriptor = data_scaler.shape[1]

In [5]:
# for pearson
Threshold = 0.00087
pearson_correlation_list = []
pearson_pvalue_list = []
pearson_selection_list = []
pearson_list = []
for i in range(1,column_descriptor):
    pearson_correlation_list.append(scipy.stats.pearsonr(data_scaler[:,i],data_scaler[:,0])[0])
    pearson_pvalue_list.append(scipy.stats.pearsonr(data_scaler[:,i],data_scaler[:,0])[1])
    if pearson_pvalue_list[i-1] > Threshold:
        pearson_selection_list.append(0)
    else :
        pearson_selection_list.append(1)
pearson_list.append(pearson_correlation_list)
pearson_list.append(pearson_pvalue_list)
pearson_list.append(pearson_selection_list)

In [6]:
# for spearman
Threshold = 1.1319835E-07
spearman_correlation_list = []
spearman_pvalue_list = []
spearman_selection_list = []
spearman_list = []
for i in range(1,column_descriptor):
    spearman_correlation_list.append(scipy.stats.spearmanr(data_scaler[:,i],data_scaler[:,0])[0])
    spearman_pvalue_list.append(scipy.stats.spearmanr(data_scaler[:,i],data_scaler[:,0])[1])
    if spearman_pvalue_list[i-1] > Threshold:
        spearman_selection_list.append(0)
    else :
        spearman_selection_list.append(1)
spearman_list.append(spearman_correlation_list)
spearman_list.append(spearman_pvalue_list)
spearman_list.append(spearman_selection_list)

In [7]:
# for distance
Threshold = 0.3020
distance_correlation_list = []
distance_selection_list = []
distance_list = []
for i in range(1,column_descriptor):
    distance_correlation_list.append(distcorr(data_scaler[:,i],data_scaler[:,0]))
    if abs(distance_correlation_list[i-1]) <= Threshold:
        distance_selection_list.append(0)
    else :
        distance_selection_list.append(1)
distance_list.append(distance_correlation_list)
distance_list.append(distance_selection_list)

In [8]:
# for MIC
Threshold = 0.0
mic_correlation_list = []
mic_selection_list = []
mic_list = []
for i in range(1,column_descriptor):
    mine.compute_score(data_scaler[:,i],data_scaler[:,0])
    mic_correlation_list.append(mine.mic())
    if abs(mic_correlation_list[i-1]) <= Threshold:
        mic_selection_list.append(0)
    else:
        mic_selection_list.append(1)
mic_list.append(mic_correlation_list)
mic_list.append(mic_selection_list)

In [9]:
# results
sum_list = []
for j in range(0,descriptor_count):
    sum_list.append(pearson_selection_list[j] + spearman_selection_list[j] + distance_selection_list[j] + mic_selection_list[j])

sum_selection_list1 = []
Threshold1 = 1
for j in range(0,descriptor_count):
    if sum_list[j] >= Threshold1:
        sum_selection_list1.append(1)
    else:
        sum_selection_list1.append(0)
# sum(sum_selection_list1)

sum_selection_list2 = []
Threshold2 = 2
for j in range(0,descriptor_count):
    if sum_list[j] >= Threshold2:
        sum_selection_list2.append(1)
    else:
        sum_selection_list2.append(0)
# sum(sum_selection_list2)

sum_selection_list3 = []
Threshold3 = 3
for j in range(0,descriptor_count):
    if sum_list[j] >= Threshold3:
        sum_selection_list3.append(1)
    else:
        sum_selection_list3.append(0)
# sum(sum_selection_list3)

sum_selection_list4 = []
Threshold4 = 4
for j in range(0,descriptor_count):
    if sum_list[j] >= Threshold4:
        sum_selection_list4.append(1)
    else:
        sum_selection_list4.append(0)
# sum(sum_selection_list4)

In [10]:
sum_list_all = []
sum_list_all.append(sum_selection_list1)
sum_list_all.append(sum_selection_list2)
sum_list_all.append(sum_selection_list3)
sum_list_all.append(sum_selection_list4)
sum_list_all.append(sum_list)
selection_list_all = []
selection_list_all.append(descriptor_name_list)
selection_list_all.append(pearson_list[0])
selection_list_all.append(pearson_list[1])
selection_list_all.append(pearson_list[2])
selection_list_all.append(spearman_list[0])
selection_list_all.append(spearman_list[1])
selection_list_all.append(spearman_list[2])
selection_list_all.append(distance_list[0])
selection_list_all.append(distance_list[1])
selection_list_all.append(mic_list[0])
selection_list_all.append(mic_list[1])
selection_list_all.append(sum_list_all[0])
selection_list_all.append(sum_list_all[1])
selection_list_all.append(sum_list_all[2])
selection_list_all.append(sum_list_all[3])
selection_list_all.append(sum_list_all[4])
selection_list_all = pd.DataFrame(selection_list_all)

selection_list_all_transpose = selection_list_all.T
selection_list_all_transpose.rename(columns={0:'descriptor_name',1:'pearson_correlation',2:'pearson_pvalue',3:'pearson_selection',
                                             4:'spearman_correlation',5:'spearman_pvalue',6:'spearman_selection',7:'distance_correlation',
                                             8:'distance_selection',9:'mic_correlation',10:'mic_selection',11:'sum_1',
                                             12:'sum_2',13:'sum_3',14:'sum_4',15:'sum'},inplace=True)

filter_data1 = all_data
for k in range(0,len(sum_selection_list1)):
    if sum_selection_list1[k] == 0:
        filter_data1 = filter_data1.drop(descriptor_name_list[k],axis=1)

filter_data2 = all_data
for k in range(0,len(sum_selection_list2)):
    if sum_selection_list2[k] == 0:
        filter_data2 = filter_data2.drop(descriptor_name_list[k],axis=1)

filter_data3 = all_data
for k in range(0,len(sum_selection_list3)):
    if sum_selection_list3[k] == 0:
        filter_data3 = filter_data3.drop(descriptor_name_list[k],axis=1)

filter_data4 = all_data
for k in range(0,len(sum_selection_list4)):
    if sum_selection_list4[k] == 0:
        filter_data4 = filter_data4.drop(descriptor_name_list[k],axis=1)

selection_list_all_transpose.to_csv('selection_list_all_transpose.csv')
filter_data1.to_csv('filter_data1.csv')
filter_data2.to_csv('filter_data2.csv')
filter_data3.to_csv('filter_data3.csv')
filter_data4.to_csv('filter_data4.csv')